### 지도 학습 - 분류모델 추가 학습 문제2
- 신용카드 사기 거래 탐지
- 신용카드 사기 거래 탐지는 이진 분류 문제로, 주어진 거래가 정상인지 사기인지 예측
- KNeighborsClassifier 알고리즘 이용

- 구현 가이드
    - **데이터 로드**: 신용카드 거래 데이터셋을 로드
    - 데이터 분할: 데이터를 학습 세트(80%)와 테스트 세트(20%)로 분할
    - 데이터 스케일링: KNN은 거리 기반 알고리즘이므로, 특성 값의 범위가 다르면 성능에 영향을 줄 수 있습니다. 이를 해결하기 위해 StandardScaler를 사용하여 데이터를 표준화
    - KNN 모델 학습: KNeighborsClassifier를 사용하여 KNN 분류 모델을 생성하고 학습 데이터를 학습시킵니다. n_neighbors=5는 가장 가까운 5개의 이웃을 기준으로 분류
    - 예측 및 성능 평가: 학습된 모델로 테스트 데이터를 예측하고, accuracy_score와 classification_report로 성능을 평가

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [10]:
#데이터 가져오기
df = pd.read_csv('creditcard.csv')

In [11]:
#특성과 레이블 분리
X = df.drop(columns='Class') #특성
y = df['Class'] #타깃

In [12]:
#데이터 분할 8:2
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y, 
    test_size=0.2, 
    random_state=42)

In [13]:
#데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [14]:
#KNN 모델 사용
k = 3   #가장 가까운 이웃의 개수
knn = KNeighborsClassifier(n_neighbors=k)


In [15]:
#모델 학습 -> 평가
knn.fit(X_train_scaled, y_train) #모델 학습
y_pred = knn.predict(X_test_scaled) # 테스트 데이터로 예측
acc = accuracy_score(y_test, y_pred) # 정확도 계산

print(f'KNN모델 정확도(K = {k}): {acc * 100}%')


KNN모델 정확도(K = 3): 99.95435553526913%


In [16]:
#분류 보고서
print('분류 보고서')
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraud']))

분류 보고서
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     56864
       Fraud       0.95      0.78      0.85        98

    accuracy                           1.00     56962
   macro avg       0.97      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962



In [18]:
accuracies = {} # K 값별 정확도를 저장할 딕셔너리

In [23]:
for k in range(1, 21): # K 값을 1부터 20까지 테스트
    knn = KNeighborsClassifier(n_neighbors=k) # KNN 모델 생성
    knn.fit(X_train_scaled, y_train) # 모델 학습
    y_pred = knn.predict(X_test_scaled) # 예측
    accuracy = accuracy_score(y_test, y_pred) # 정확도 계산
    accuracies[k] = accuracy # K 값별 정확도 저장

In [24]:
#결과 출력
for k, acc in accuracies.items():
    print(f"K={k}: 정확도={acc * 100:.2f}%")

K=1: 정확도=99.94%
K=2: 정확도=99.95%
K=3: 정확도=99.95%
K=4: 정확도=99.95%
K=5: 정확도=99.95%
K=6: 정확도=99.95%
K=7: 정확도=99.95%
K=8: 정확도=99.94%
K=9: 정확도=99.94%
K=10: 정확도=99.94%
K=11: 정확도=99.94%
K=12: 정확도=99.94%
K=13: 정확도=99.94%
K=14: 정확도=99.94%
K=15: 정확도=99.94%
K=16: 정확도=99.94%
K=17: 정확도=99.94%
K=18: 정확도=99.94%
K=19: 정확도=99.94%
K=20: 정확도=99.94%


In [25]:
#최적의 K 값 찾기
best_k = max(accuracies, key=accuracies.get) # 정확도가 가장 높은 K 값
print(f"\n최적의 K 값: {best_k}, 정확도: {accuracies[best_k] * 100:.2f}%")


최적의 K 값: 3, 정확도: 99.95%


In [26]:
#거리 계산 방식 변경 및 가중치 추가
#최적의 K 값과 다른 거리 계산 방식을 사용하여 모델 생성
final_knn = KNeighborsClassifier(
    n_neighbors=best_k, 
    weights='distance',
    metric='manhattan') #가중치!

final_knn.fit(X_train_scaled, y_train) # 최적의 하이퍼파라미터로 모델 학습


,n_neighbors,3
,weights,'distance'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'manhattan'
,metric_params,None
,n_jobs,None


In [27]:
#최적화된 모델 평가
y_pred = final_knn.predict(X_test_scaled) # 테스트 데이터로 예측
accuracy = accuracy_score(y_test, y_pred) # 정확도 계산
print(f"최적화된 KNN 모델 정확도: {accuracy * 100:.2f}%")

최적화된 KNN 모델 정확도: 99.95%


In [28]:
#세부적인 성능 보고서 출력
print("\n분류 보고서:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Fraud']))


분류 보고서:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     56864
       Fraud       0.96      0.77      0.85        98

    accuracy                           1.00     56962
   macro avg       0.98      0.88      0.93     56962
weighted avg       1.00      1.00      1.00     56962

